In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="NorthCorp")
import datetime
import matplotlib.pyplot as plt
import os

In [5]:
base_dir = r"/home/parth/Machine Learning/Datasets/North Corp/Acid"
files = os.listdir(base_dir)
files_corrected = []
data = []
first_one = True
for file in files :
    if file.endswith(".csv") :
        files_corrected.append(file)
        dat = pd.read_csv(os.path.join( base_dir , file) , delimiter=';')
        if first_one :
            columns = dat.columns
        else :
            assert  np.all(columns == dat.columns)
        first_one = False
        data.append(dat)
        
files = files_corrected
del files_corrected

In [13]:
data[0].head()

,positionid,vehicleid,gtpdriverid,version,heading,latitude,longitude,altitude,address,updateddate,...,time_idle,time_pto,time_drive,speed,msg_trigger_type,current_fuel_level,received_date,dateSeconds,idleWastedFuel,idleTimeDelta
0,822315139,24412,NaN,0,0,18.941709,73.187177,21.0,NaN,2019-01-31 02:04:44,...,3.63,0.0,19.15,0.0,IGNITION_ON,4.4,NaN,1.548900e+09,0,0
1,822315626,24412,NaN,0,0,18.941677,73.187172,15.0,NaN,2019-01-31 02:05:36,...,3.63,0.0,19.15,0.0,IGNITION_OFF,4.4,NaN,1.548900e+09,0,None
2,822320601,24412,NaN,0,0,18.941683,73.187171,24.0,NaN,2019-01-31 02:09:42,...,3.63,0.0,19.15,0.0,PERIODIC,4.4,NaN,1.548901e+09,0,None
3,822326827,24412,NaN,0,0,18.941690,73.187161,24.0,NaN,2019-01-31 02:14:42,...,3.63,0.0,19.15,0.0,PERIODIC,4.4,NaN,1.548901e+09,0,None
4,822332977,24412,NaN,0,0,18.941695,73.187140,24.0,NaN,2019-01-31 02:19:42,...,3.63,0.0,19.15,0.0,PERIODIC,4.4,NaN,1.548901e+09,0,None


In [7]:
for i in range(len(data)) :
    data[i] = data[i].sort_values("updateddate" , ascending=True)

for i in range(len(data)) :
    data[i]["dateSeconds"] = data[i]["updateddate"].apply(
        lambda x: datetime.datetime.strptime(x , "%Y-%m-%d %H:%M:%S")).apply(
        lambda x: (x - datetime.datetime(1970,1,1)).total_seconds() )

In [8]:
for i in range(len(data)) :
    previdletime = data[i].iloc[0]["time_idle"]
    data[i]["idleWastedFuel"] = None
    data[i]["idleTimeDelta"] = None
    for row in range(1 , len(data[i])):
        currentidletime = data[i].iloc[row]["time_idle"]
        currentFuel = data[i].iloc[row]["total_fuel_consumption"]
        if (currentidletime != previdletime):
            if row != 0 :
                data[i].at[row , "idleWastedFuel"] = abs ((currentFuel - prevFuel))
                data[i].at[row  , "idleTimeDelta"] = abs(currentidletime - previdletime)
            else :
                data[i].at[row , "idleWastedFuel"] = 0
        else:
            data[i].at[row , "idleWastedFuel"] = 0
        previdletime = currentidletime
        prevFuel = currentFuel
    data[i].at[0 , "idleWastedFuel"] = 0
    data[i].at[0 , "idleTimeDelta"] = 0

In [9]:
IdleCases = []

for i in range(len(data)) :
    IdleCases.append(data[i][data[i].speed == 0])
    IdleCases[i] = IdleCases[i][IdleCases[i]["idleWastedFuel"] != 0]

In [10]:
StopCases = []

for i in range(len(data)) :
    StopCases.append(data[i][data[i].speed == 0])
    StopCases[i] = StopCases[i][StopCases[i]["idleWastedFuel"] == 0 ]
    StopCases[i]["stopTimeDelta"] = 0
    StopCases[i].drop(columns=["idleWastedFuel" , "idleTimeDelta"] , inplace=True)
    StopCases[i].stopTimeDelta = StopCases[i].dateSeconds.diff()
    StopCases[i].at[0 , "stopTimeDelta"] = 0
    StopCases[i]["stopTimeDelta"] = StopCases[i]["stopTimeDelta"] / 3600.0

In [11]:
for i in range(len(data)) :
    print("Total Fuel Wasted CASE:{} : {} ".format(i , sum(IdleCases[i]["idleWastedFuel"] )))

Total Fuel Wasted CASE:0 : 233.73999999999245 
Total Fuel Wasted CASE:1 : 81.08999999999952 
Total Fuel Wasted CASE:2 : 206.40000000000842 
Total Fuel Wasted CASE:3 : 250.2699999999957 
Total Fuel Wasted CASE:4 : 8099.8500000000095 
Total Fuel Wasted CASE:5 : 147.35000000000127 
Total Fuel Wasted CASE:6 : 15.929999999999836 
Total Fuel Wasted CASE:7 : 172.79999999999956 
Total Fuel Wasted CASE:8 : 151.35999999999316 
Total Fuel Wasted CASE:9 : 4101.189999999998 


In [12]:
for i in range(len(data)) :
    IdleCases[i].to_csv("IdleCases-{}.csv".format(files[i]))
    StopCases[i].to_csv("StopCases-{}.csv".format(files[i]))
    IdleCases[i].sort_values("idleWastedFuel" , ascending=False)[:10].to_csv("Idle Top 10 {}.csv".format(files[i]))
    data[i].to_csv("Full-Data-{}.csv".format(files[i]))